# Evaluate model

## Imports

In [1]:
import pathlib
import re
import tempfile
import shutil

import plotly.express as px
import plotly.graph_objects as go

import typing as t
import pandas as pd
import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras import metrics
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing import image

2021-10-24 18:11:47.769761: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


## Config

In [2]:
MODEL_PATH = pathlib.Path(
    "/media/cicheck/Extreme Pro/models/best_models/best_mod_2_meso_net_04_0.12.h5"
)
DATASET_PATH = pathlib.Path("/media/cicheck/Extreme Pro/datasets/modified/mod_2")

IMAGE_SIZE = (256, 256)
BATCH_SIZE = 32

## Utils

In [3]:
_metric_names_to_rephrased_names = {
    "binary_accuracy": "accuracy",
    "auc": "AUC",
    "precision": "precision",
    "recall": "recall",
    "true_positives": "TP",
    "true_negatives": "TN",
    "false_positives": "FP",
    "false_negatives": "FN",
}

def rephrase_metrics_dict(metrics_dict: dict[str, float]) -> dict[str, float]:
    # Remove sufixes, e.g. auc_3 -> auc
    prefix_pattern = re.compile(r"(?P<metric_prefix>\w+)_(?P<metric_number>\d+)")
    # TODO: ugly comperhension
    normalized_metrics_dict = {
        (prefix_pattern.match(metric).group("metric_prefix") if prefix_pattern.match(metric) else metric): value
        for metric, value in metrics_dict.items()
    }
    rephrased_metrics_dict = {
        (_metric_names_to_rephrased_names[metric] if metric in _metric_names_to_rephrased_names else metric): value
        for metric, value in normalized_metrics_dict.items()
    }
    no_tp = rephrased_metrics_dict["TP"]
    no_tn = rephrased_metrics_dict["TN"]
    no_fp = rephrased_metrics_dict["FP"]
    no_fn = rephrased_metrics_dict["FN"]
    
    no_p = no_tp + no_fn
    no_n = no_tn + no_fp
    # Represent TP, TN, FP, FN as ratio
    rephrased_metrics_dict["TP"] = no_tp / no_p if no_p != 0 else 0
    rephrased_metrics_dict["FN"] = no_fn / no_p if no_p != 0 else 0
    rephrased_metrics_dict["TN"] = no_tn / no_n if no_n != 0 else 0
    rephrased_metrics_dict["FP"] = no_fp / no_n if no_n != 0 else 0
    # Share of positives correctly recognized by model (AKA True positive rate)
#     rephrased_metrics_dict["TPR"] = (
#         rephrased_metrics_dict["TP"]
#         / (rephrased_metrics_dict["TP"] + rephrased_metrics_dict["FN"])
#     )
    # Share of negatives correctly recognized by model (AKA True negative rate)
#     rephrased_metrics_dict["TNR"] = (
#         rephrased_metrics_dict["TN"]
#         / (rephrased_metrics_dict["TN"] + rephrased_metrics_dict["FP"])
#     )
    return rephrased_metrics_dict

def _build_single_row(*columns, width):
    row_as_string = "|"
    for column in columns:
        row_as_string += column.ljust(width, " ")
        row_as_string += "|"
    row_as_string +="\n"
    return row_as_string

def print_metrics_dict_as_table(
    metrics_dict: dict[str, float], width: int = 14
):
    """Print metrics dictionay as markdown table.
    
    Args:
        metrics_dict: metrics dictionay (i.e. output of keras evaluate which output type set to dict).
        width: width of 
    """
    first_column_name = " metrics"
    second_column_name = " values"
    table_as_string = ""
    table_as_string += _build_single_row(first_column_name, second_column_name, width=width)
    header_sign = ":---:"
    table_as_string += _build_single_row(header_sign, header_sign, width=width)
    rephrased_metric_dict = rephrase_metrics_dict(metrics_dict)
    for metric, value in rephrased_metric_dict.items():
        table_as_string += _build_single_row(metric, "{:.5f}".format(value), width=width)
    print(table_as_string)
        

## Setup model

In [4]:
evaluated_model = models.load_model(MODEL_PATH, compile=False)

2021-10-24 18:11:49.147676: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-10-24 18:11:49.202609: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-24 18:11:49.202907: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 3060 Laptop GPU computeCapability: 8.6
coreClock: 1.425GHz coreCount: 30 deviceMemorySize: 5.80GiB deviceMemoryBandwidth: 312.97GiB/s
2021-10-24 18:11:49.202988: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-10-24 18:11:49.204540: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2021-10-24 18:11:49.204651: I tensorflow/stream_e

In [5]:
METRICS = [
        metrics.BinaryAccuracy(),
        metrics.AUC(),
        metrics.Precision(),
        metrics.Recall(),
        metrics.TruePositives(),
        metrics.FalseNegatives(),
        metrics.TrueNegatives(),
        metrics.FalsePositives(),
    ]

In [6]:
# Model needs to be compiled again to set targeted metrices
optimizer = optimizers.Adam(
    learning_rate=1e-3,
    epsilon=1e-08
)

evaluated_model.compile(
    optimizer=optimizer,
    loss="binary_crossentropy",
    metrics=METRICS
)

## Evaluate whole dataset

In [7]:
evaluation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    DATASET_PATH.joinpath("test"),
    batch_size=BATCH_SIZE,
    image_size=IMAGE_SIZE,
    label_mode="binary",
    class_names=["reals", "fakes"],
)

Found 468664 files belonging to 2 classes.


In [8]:
metrics_dict = evaluated_model.evaluate(evaluation_dataset, return_dict=True)

2021-10-23 15:15:56.635509: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-10-23 15:15:56.665459: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 3193965000 Hz
2021-10-23 15:15:57.173543: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudnn.so.8
2021-10-23 15:15:57.833230: I tensorflow/stream_executor/cuda/cuda_dnn.cc:359] Loaded cuDNN version 8100
2021-10-23 15:15:58.873226: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11


    3/14646 [..............................] - ETA: 10:43 - loss: 0.1099 - binary_accuracy: 0.9896 - auc: 0.8933 - precision: 0.9889 - recall: 1.0000 - true_positives: 89.0000 - false_negatives: 0.0000e+00 - true_negatives: 6.0000 - false_positives: 1.0000   

2021-10-23 15:15:59.499693: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11
2021-10-23 15:15:59.585928: I tensorflow/stream_executor/cuda/cuda_blas.cc:1838] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


  101/14646 [..............................] - ETA: 8:57 - loss: 0.1154 - binary_accuracy: 0.9619 - auc: 0.9664 - precision: 0.9721 - recall: 0.9859 - true_positives: 2859.0000 - false_negatives: 41.0000 - true_negatives: 250.0000 - false_positives: 82.0000

KeyboardInterrupt: 

In [ ]:
print_metrics_dict_as_table(metrics_dict)

In [13]:
# Fajnie było widać że jedyne co się zmieniło to niektóre true positives przeszły na false negatives

## Display scores for modifications separately

In [8]:
modifications_dir = DATASET_PATH / "test" / "reals"

In [9]:
for modification_dir in modifications_dir.iterdir():
    no_mod_images = len(list(modification_dir.rglob("*.png")))
    # Copy selected modification dir into temp dirsince keras method
    # image_dataset_from_directory requires images to lay in subdirectory
    with tempfile.TemporaryDirectory() as tmp_dir_path:
        shutil.copytree(
            modification_dir, pathlib.Path(tmp_dir_path).joinpath(modification_dir.name)
        )
        modifications_ds =  tf.keras.preprocessing.image_dataset_from_directory(
            tmp_dir_path,
            labels=list(0 for _ in range(no_mod_images)),
            batch_size=BATCH_SIZE,
            image_size=IMAGE_SIZE,
        )
        metrics_dict = evaluated_model.evaluate(modifications_ds, return_dict=True)
        print(modification_dir.name)
        print_metrics_dict_as_table(metrics_dict)

Found 3690 files belonging to 1 classes.


2021-10-19 16:07:03.391410: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-10-19 16:07:03.405733: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 3193880000 Hz
2021-10-19 16:07:03.693327: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudnn.so.8
2021-10-19 16:07:04.170924: I tensorflow/stream_executor/cuda/cuda_dnn.cc:359] Loaded cuDNN version 8100
2021-10-19 16:07:04.874823: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11


  4/116 [>.............................] - ETA: 2s - loss: 0.0595 - binary_accuracy: 0.9688 - auc: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00 - true_positives: 0.0000e+00 - false_negatives: 0.0000e+00 - true_negatives: 124.0000 - false_positives: 4.0000 

2021-10-19 16:07:05.283176: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11
2021-10-19 16:07:05.366990: I tensorflow/stream_executor/cuda/cuda_blas.cc:1838] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


116/116 [==============================] - 5s 22ms/step - loss: 0.1137 - binary_accuracy: 0.9566 - auc: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00 - true_positives: 0.0000e+00 - false_negatives: 0.0000e+00 - true_negatives: 3530.0000 - false_positives: 160.0000
histogram_equalization
| metrics      | values       |
|:---:         |:---:         |
|loss          |0.11371       |
|accuracy      |0.95664       |
|AUC           |0.00000       |
|precision     |0.00000       |
|recall        |0.00000       |
|TP            |0.00000       |
|FN            |0.00000       |
|TN            |0.95664       |
|FP            |0.04336       |

Found 3690 files belonging to 1 classes.
116/116 [==============================] - 3s 21ms/step - loss: 0.0961 - binary_accuracy: 0.9691 - auc: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00 - true_positives: 0.0000e+00 - false_negatives: 0.0000e+00 - true_negatives: 3576.0000 - false_positives: 114.0000
clahe
| metrics      | values    

## Evaluate modifications separately

In [50]:
## Utils
def get_gamma_correction_params(dir_name: str) -> t.Dict[str, t.Union[int, float]]:
    dir_name_pattern = re.compile(
        r"""(?P<modification_name>[a-z_]+)_
        (?P<gamma_value>\d+\.\d+)
        """,
        flags=re.VERBOSE
    )
    gamma_value = float(dir_name_pattern.match(dir_name).group("gamma_value"))
    return {"gamma_value": gamma_value}

def get_red_eyes_effect_params(dir_name: str) -> t.Dict[str, t.Union[int, float]]:
    dir_name_pattern = re.compile(
        r"""(?P<modification_name>[a-z_]+)_
        (?P<brightness_threshold>\d+)
        """,
        flags=re.VERBOSE
    )
    brightness_threshold = float(dir_name_pattern.match(dir_name).group("brightness_threshold"))
    return {"brightness_threshold": brightness_threshold}

def get_gaussian_blur_params(dir_name: str) -> t.Dict[str, t.Union[int, float]]:
    dir_name_pattern = re.compile(
        r"""(?P<modification_name>[a-z_]+)
        (?P<kernel_width>\d+)_
        (?P<kernel_height>\d+)_
        (?P<sigma_x>\d+)_
        (?P<sigma_y>\d+)
        """,
        flags=re.VERBOSE
    )
    match = dir_name_pattern.match(dir_name)
    kernel_width = int(match.group("kernel_width"))
    kernel_height = int(match.group("kernel_height"))
    sigma_x = int(match.group("sigma_x"))
    sigma_y = int(match.group("sigma_y"))
    return {
        "kernel_width": kernel_width,
        "kernel_height": kernel_height,
        "sigma_x": sigma_x,
        "sigma_y": sigma_y,
    }

def get_clahe_params(dir_name: str) -> t.Dict[str, t.Union[int, float]]:
    dir_name_pattern = re.compile(
        r"""(?P<modification_name>[a-z_]+)_
        (?P<grid_width>\d+)_
        (?P<grid_height>\d+)_
        (?P<clip_limit>\d+\.\d+)
        """,
        flags=re.VERBOSE
    )
    match = dir_name_pattern.match(dir_name)
    grid_width = int(dir_name_pattern.match(dir_name).group("grid_width"))
    grid_height = int(dir_name_pattern.match(dir_name).group("grid_height"))
    clip_limit = float(dir_name_pattern.match(dir_name).group("clip_limit"))
    
    return {
        "grid_width": grid_width,
        "grid_height": grid_height,
        "clip_limit": clip_limit,
    }

def get_median_filter_params(dir_name: str) -> t.Dict[str, t.Union[int, float]]:
    dir_name_pattern = re.compile(
        r"""(?P<modification_name>[a-z_]+)_
        (?P<aperture_size>\d+)
        """,
        flags=re.VERBOSE
    )
    aperture_size = int(dir_name_pattern.match(dir_name).group("aperture_size"))
    return {"aperture_size": aperture_size}

In [8]:
BASE_PATH = pathlib.Path("/media/cicheck/Extreme Pro/datasets/modified/test_modifications")

In [40]:
def load_modification(
    modifications_test_ds_path: pathlib.Path,
    modification_params_reader: t.Callable[[str], dict],
) -> pd.DataFrame:
    records = []

    for modification_dir in modifications_test_ds_path.iterdir():
        no_mod_images = len(list(modification_dir.iterdir()))
        # Copy selected modification dir into temp dirsince keras method
        # image_dataset_from_directory requires images to lay in subdirectory
        with tempfile.TemporaryDirectory() as tmp_dir_path:
            shutil.copytree(
                modification_dir,
                pathlib.Path(tmp_dir_path).joinpath(modification_dir.name),
            )
            modifications_ds =  tf.keras.preprocessing.image_dataset_from_directory(
                tmp_dir_path,
                labels=list(0 for _ in range(no_mod_images)),
                batch_size=BATCH_SIZE,
                image_size=IMAGE_SIZE,
            )
            metrics_dict = evaluated_model.evaluate(
                modifications_ds, return_dict=True, verbose=0
            )
            rephrased_metrics_dict = rephrase_metrics_dict(metrics_dict)
            records.append(
                {
                    **modification_params_reader(modification_dir.name),
                    "true_negative_rate": rephrased_metrics_dict["TN"],
                }
            )
    return pd.DataFrame(records)

In [57]:
def plot_gamma_modification_scores(dataframe: pd.DataFrame) -> go.Figure:
    dataframe = dataframe.sort_values("gamma_value")
    fig = px.line(
        dataframe,
        x="gamma_value",
        y="true_negative_rate",
        markers=True,
        line_shape="linear",
        labels={
            "gamma_value": "Gamma value",
            "true_negative_rate": "True Negative Rate",
        },
        title="Gamma correction",
    )
    fig.update_layout(
        xaxis = dict(
            tickmode = "linear",
            tick0 = 0.1,
            dtick = 0.1,
        )
    )
    return fig

def plot_gaussian_blur_scores(dataframe: pd.DataFrame) -> go.Figure:
    dataframe = dataframe.sort_values("kernel_width")
    fig = px.line(
        dataframe,
        x="kernel_width",
        y="true_negative_rate",
        markers=True,
        line_shape="linear",
        labels={
            "kernel_width": "Kernel Size",
            "true_negative_rate": "True Negative Rate",
        },
        title="Gaussian blur",
    )
    fig.update_layout(
        xaxis = dict(
            tickmode = "linear",
            tick0 = 1,
            dtick = 2,
        )
    )
    return fig

def plot_median_filter(dataframe: pd.DataFrame) -> go.Figure:
    dataframe = dataframe.sort_values("aperture_size")
    fig = px.line(
        dataframe,
        x="aperture_size",
        y="true_negative_rate",
        markers=True,
        line_shape="linear",
        labels={
            "aperture_size": "Aperture size",
            "true_negative_rate": "True Negative Rate",
        },
        title="Median filter",
    )
    fig.update_layout(
        xaxis = dict(
            tickmode = "linear",
            tick0 = 1,
            dtick = 2,
        )
    )
    return fig

def plot_clahe(dataframe: pd.DataFrame) -> go.Figure:
    dataframe=dataframe.sort_values(["grid_width", "clip_limit"])
    fig = px.line(
        dataframe,
        x="clip_limit",
        y="true_negative_rate",
        facet_col="grid_width",
        color="grid_width",
        labels={
            "clip_limit": "Clip limit",
            "true_negative_rate": "True Negative Rate",
            "grid_width": "Kernel size"
        },
        markers=True,
        title="CLAHE",
    )
    fig.for_each_xaxis(
        lambda xaxis: xaxis.update(
                tickmode = "linear",
                tick0 = 1.0,
                dtick = 1.0,
            ),
    )
    return fig

def plot_clahe_3d(dataframe: pd.DataFrame) -> go.Figure:
    dataframe=dataframe.sort_values(["grid_width", "clip_limit"])
    fig = px.line_3d(
        dataframe,
        x="clip_limit",
        y="true_negative_rate",
        z="grid_width",
        labels={
            "clip_limit": "Clip limit",
            "true_negative_rate": "True Negative Rate",
            "grid_width": "Kernel size"
        },
        markers=True,
        title="CLAHE",
    )
    fig.update_layout(
        scene=dict(
            xaxis = dict(
                tickmode = "linear",
                tick0 = 1.0,
                dtick = 1.0,
            ),
            zaxis = dict(
                tickmode = "linear",
                tick0 = 0,
                dtick = 8,
                range = [0, 32],
            ),
        )
    )
    return fig
    

In [35]:
images_path = pathlib.Path("/media/cicheck/Extreme Pro/images")

In [46]:
modifications_test_ds_path = BASE_PATH / "test_gamma_correction" / "reals"
modification_df = load_modification(
    modifications_test_ds_path,
    get_gamma_correction_params,
)

fig = plot_gamma_modification_scores(modification_df)
fig.write_image(images_path / "gamma_correction.svg")
fig

Found 2306 files belonging to 1 classes.
Found 2306 files belonging to 1 classes.
Found 2306 files belonging to 1 classes.
Found 2307 files belonging to 1 classes.
Found 2306 files belonging to 1 classes.
Found 2306 files belonging to 1 classes.
Found 2306 files belonging to 1 classes.
Found 2307 files belonging to 1 classes.
Found 2306 files belonging to 1 classes.
Found 2306 files belonging to 1 classes.
Found 2306 files belonging to 1 classes.
Found 2307 files belonging to 1 classes.
Found 2306 files belonging to 1 classes.
Found 2306 files belonging to 1 classes.
Found 2306 files belonging to 1 classes.
Found 2307 files belonging to 1 classes.
Found 2306 files belonging to 1 classes.
Found 2306 files belonging to 1 classes.
Found 2306 files belonging to 1 classes.
Found 2307 files belonging to 1 classes.


In [37]:
modifications_test_ds_path = BASE_PATH / "test_gaussian_blur" / "reals"
modification_df = load_modification(
    modifications_test_ds_path,
    get_gaussian_blur_params,
)

fig = plot_gaussian_blur_scores(modification_df)
fig.write_image(images_path / "gaussian_blur.svg")
fig

In [45]:
modifications_test_ds_path = BASE_PATH / "test_median_filter" / "reals"
modification_df = load_modification(
    modifications_test_ds_path,
    get_median_filter_params,
)

fig = plot_median_filter(modification_df)
fig.write_image(images_path / "median_filter.svg")
fig

In [85]:
modifications_test_ds_path = BASE_PATH / "test_clache" / "reals"
modification_df = load_modification(
    modifications_test_ds_path,
    get_clahe_params,
)

fig = plot_clahe(modification_df)
fig.write_image(images_path / "clahe.svg")
